In [ ]:
%load_ext slurm_magic

In [ ]:
%%sbatch
#!/bin/bash
#SBATCH --constraint=haswell
#SBATCH --nodes=2
#SBATCH --partition=debug
#SBATCH --time=30

module load python/3.6-anaconda-5.2

# Get IP address of head node
head_ip=$(ip addr show ipogif0 | grep '10\.' | awk '{print $2}' | awk -F'/' '{print $1}')

# Unique cluster ID for this job
cluster_id=cori_${SLURM_JOB_ID}

# Cluster controller
ipcontroller --ip="$head_ip" --cluster-id=$cluster_id &
sleep 10

# Compute engines
srun -u -n 32 ipengine --cluster-id=$cluster_id

In [ ]:
job_id = _.split()[-1]
job_id

In [ ]:
cluster_id = "cori_" + job_id
cluster_id

In [ ]:
%squeue -u rthomas

In [ ]:
import ipyparallel as ipp

In [ ]:
c = ipp.Client(timeout=60, cluster_id=cluster_id)

In [ ]:
c.ids

In [ ]:
%%px
from mpi4py import MPI

In [ ]:
%%px
print(MPI.COMM_WORLD.rank)

In [ ]:
%%px
import random
import math

In [ ]:
%%px
def work(trials):
    count = 0.0
    for i in range(trials):
        x = random.random()
        y = random.random()
        if x ** 2 + y ** 2 < 1.0:
            count += 1.0
    return count

In [ ]:
%%px
def compute_pi(trials):
    
    # Initialize MPI
    
    mpi_comm = MPI.COMM_WORLD
    mpi_size = mpi_comm.size
    mpi_rank = mpi_comm.rank
    mpi_root = mpi_rank == 0
    
    # Initialize random number generator
    
    random.seed(192837465 + mpi_rank)
    
    # Distribute work, gather results, and report answer
    
    count = work(trials)
    count = mpi_comm.gather(count, root=0)
    if mpi_root:
        total_count = sum(count)
        total_trials = mpi_size * trials
        estimated_pi = 4.0 * total_count / total_trials
        print("Total Count:   ", total_count)
        print("Total Trials:  ", total_trials)
        print("Estimate of pi:", estimated_pi, math.pi, abs(estimated_pi - math.pi)) 

In [ ]:
%%px
compute_pi(10000000)

In [ ]:
%%px --targets 0
work(10000000)

In [ ]:
%scancel $job_id